In [1]:
import sys
from PyQt5.QtWidgets import *
from PyQt5 import uic

import serial
import time
import sys
import glob
import win32com.client


s = serial.Serial('COM11',9600)
form_class = uic.loadUiType("main_window.ui")[0]
read_list = []
excel = win32com.client.Dispatch("Excel.Application")

In [2]:
def _twos_comp(val, bits):
    # Convert an unsigned integer in 2's compliment form of the specified bit
    # length to its signed integer value and return it.
    if val & (1 << (bits - 1)) != 0:
        return val - (1 << bits)
    return val

In [3]:
class MyWindow(QMainWindow, form_class):
    def __init__(self):
        super().__init__()
        self.setupUi(self)
        self.pushButton.clicked.connect(self.btn_clicked)
        self.pushButton_2.clicked.connect(self.btn_close)
        self.pushButton_3.clicked.connect(self.btn_start)
        self.pushButton_4.clicked.connect(self.btn_showData)
        self.pushButton_5.clicked.connect(self.btn_saveExcel)

    def btn_clicked(self):
       
        result = s.is_open
       
        if result == True:
            QMessageBox.about(self, "message", "Connected")
            self.textEdit.setText("연결 성공")
        else:
            QMessageBox.about(self, "message", "Error")
            
    def btn_close(self):
        s.close()
        QMessageBox.about(self, "message", "연결 종료")
        self.textEdit.setText("연결 종료")
        
    def btn_start(self):
        
        self.textEdit.setText("전송 시작")
        s.write('s'.encode())
        while(True):
            read = s.readline()
            read = read.decode()
            read_list.append(read)
            print(read)
            #time.sleep(1)
            
            if(read[0] == 'e'):
                self.textEdit.setText("전송 종료")
                return 
            
    def btn_saveExcel(self):
        excel.Visible = True
        wb = excel.Workbooks.Add()
        ws = wb.Worksheets("Sheet1")
        list_length = len(read_list)
        for i in range(list_length - 1):
            k = i + 1;
            trim_list = read_list[i].split(" ")[1:-1]
            index = trim_list[0]
            timestamp = '0x' + ''.join(trim_list[1:5])
            hex_int = int(timestamp, 16)
            datetime = time.strftime('%Y-%m-%d,%H:%M:%S', time.localtime(hex_int))

            acc_x = (((int(trim_list[5], 16) << 8 | int(trim_list[6], 16))))
            acc_y = (((int(trim_list[7], 16) << 8 | int(trim_list[8], 16))))
            acc_z = (((int(trim_list[9], 16) << 8 | int(trim_list[10], 16))))

            a = _twos_comp(acc_x >> 2 , 14) * 0.000488
            b = _twos_comp(acc_y >> 2 , 14) * 0.000488
            c = _twos_comp(acc_z >> 2 , 14) * 0.000488
            
            result_string = index + "," + datetime + "," +  '{0:0.3f},{1:0.3f},{2:0.3f}'.format(a, b, c )
            self.textEdit.append(result_string)
            print(result_string , type(result_string))
            
            ws.Cells(k, 1).Value = index
            ws.Cells(k, 2).Value = datetime
            ws.Cells(k, 3).Value = a
            ws.Cells(k, 4).Value = b
            ws.Cells(k, 5).Value = c
                
        
        #wb.SaveAs('test.xlsx')
        excel.Quit()
    def btn_showData(self):
        #s.close()
        #for 
        #QMessageBox.about(self, "message", "연결 종료")
        self.textEdit.setText("데이터 확인")
        list_length = len(read_list)
        for i in range(list_length - 1):
            
            trim_list = read_list[i].split(" ")[1:-1]
            index = trim_list[0]
        #     print(len(read_list),len(read_list[i].split(" ")[1:-1]))
            timestamp = '0x' + ''.join(trim_list[1:5])
            hex_int = int(timestamp, 16)
        #     print(hex_int)
            datetime = time.strftime('%Y-%m-%d,%H:%M:%S', time.localtime(hex_int))

            acc_x = (((int(trim_list[5], 16) << 8 | int(trim_list[6], 16))))
            acc_y = (((int(trim_list[7], 16) << 8 | int(trim_list[8], 16))))
            acc_z = (((int(trim_list[9], 16) << 8 | int(trim_list[10], 16))))

            a = _twos_comp(acc_x >> 2 , 14) * 0.000488
            b = _twos_comp(acc_y >> 2 , 14) * 0.000488
            c = _twos_comp(acc_z >> 2 , 14) * 0.000488

        #     print(a , type(a))
        #     print(b , type(b))
        #     print(c , type(c))
            result_string = index + "," + datetime + "," +  '{0:0.3f},{1:0.3f},{2:0.3f}'.format(a, b, c )
            self.textEdit.append(result_string)
            print(result_string , type(result_string))
        
        
if __name__ == "__main__":
    app = QApplication(sys.argv)
    myWindow = MyWindow()
    myWindow.show()
    app.exec_()

00000: 00 5a c1 28 10 00 00 ff 08 e0 00 00 00 00 00 00 

00010: 01 5a c1 28 10 01 20 ff 00 e2 10 00 00 00 00 00 

00020: 02 5a c1 28 11 00 00 ff 00 e2 00 00 00 00 00 00 

00030: 03 5a c1 28 10 01 08 fe 00 e0 10 00 00 00 00 00 

00040: 04 5a c1 28 12 01 00 fe 08 e0 04 00 00 00 00 00 

00050: 05 5a c1 28 12 00 00 fd 04 e0 08 00 00 00 00 00 

00060: 06 5a c1 28 12 00 00 fd 00 e2 00 00 00 00 00 00 

00070: 07 5a c1 28 00 01 40 fd 00 e2 00 00 00 00 00 00 

00080: 08 5a c1 28 00 00 00 fd 08 e0 08 00 00 00 00 00 

00090: 09 5a c1 28 00 00 00 fe 08 e2 10 00 00 00 00 00 

000a0: 0a 5a c1 28 00 01 00 fe 00 e2 08 00 00 00 00 00 

000b0: 0b 5a c1 28 00 01 00 fe 10 e2 00 00 00 00 00 00 

000c0: 0c 5a c1 28 00 01 00 fe 00 e2 00 00 00 00 00 00 

000d0: 0d 5a c1 28 01 01 00 fe 10 e2 00 00 00 00 00 00 

000e0: 0e 5a c1 28 01 01 00 ff 00 e0 04 00 00 00 00 00 

000f0: 0f 5a c1 28 00 01 40 ff 00 e0 00 00 00 00 00 ff 

00000: 10 5a c1 28 00 01 04 ff 00 e2 00 00 00 00 00 00 

00010: 11 5a c1 28 02 01 00 ff 

00010: 91 5a c1 29 bb 01 40 fe 00 e0 08 00 00 00 00 00 

00020: 92 5a c1 29 c0 01 40 ff 04 e2 08 00 00 00 00 00 

00030: 93 5a c1 29 c0 01 40 fe 0c e0 10 00 00 00 00 00 

00040: 94 5a c1 29 c0 01 40 ff 00 e0 00 00 00 00 00 00 

00050: 95 5a c1 29 c0 01 40 fe 18 e0 20 00 00 00 00 00 

00060: 96 5a c1 29 c0 01 40 fe 24 e2 00 00 00 00 00 00 

00070: 97 5a c1 29 c1 01 04 ff 00 e0 04 00 00 00 00 00 

00080: 98 5a c1 29 c1 01 40 ff 00 e2 08 00 00 00 00 00 

00090: 99 5a c1 29 c1 01 08 fe 10 e2 08 00 00 00 00 00 

000a0: 9a 5a c1 29 c1 01 30 fe f0 e2 00 00 00 00 00 00 

000b0: 9b 5a c1 29 c0 01 14 ff 00 e2 04 00 00 00 00 00 

000c0: 9c 5a c1 29 c2 01 00 ff 00 e2 04 00 00 00 00 00 

000d0: 9d 5a c1 29 c2 01 40 ff 04 e2 04 00 00 00 00 00 

000e0: 9e 5a c1 29 c2 01 40 fe 00 e2 00 00 00 00 00 00 

000f0: 9f 5a c1 29 c2 01 00 ff 10 e2 10 00 00 00 00 ff 

00000: a0 5a c1 29 c3 01 00 fe 14 e2 00 00 00 00 00 00 

00010: a1 5a c1 29 c3 01 00 fe 30 e0 48 00 00 00 00 00 

00020: a2 5a c1 29 c3 01 3c fe 

00,2018-04-02,03:42:24,0.000,-0.030,-0.999 <class 'str'>
01,2018-04-02,03:42:24,0.035,-0.031,-0.935 <class 'str'>
02,2018-04-02,03:42:25,0.000,-0.031,-0.937 <class 'str'>
03,2018-04-02,03:42:24,0.032,-0.062,-0.997 <class 'str'>
04,2018-04-02,03:42:26,0.031,-0.061,-0.999 <class 'str'>
05,2018-04-02,03:42:26,0.000,-0.093,-0.998 <class 'str'>
06,2018-04-02,03:42:26,0.000,-0.094,-0.937 <class 'str'>
07,2018-04-02,03:42:08,0.039,-0.094,-0.937 <class 'str'>
08,2018-04-02,03:42:08,0.000,-0.093,-0.998 <class 'str'>
09,2018-04-02,03:42:08,0.000,-0.061,-0.935 <class 'str'>
0a,2018-04-02,03:42:08,0.031,-0.062,-0.936 <class 'str'>
0b,2018-04-02,03:42:08,0.031,-0.061,-0.937 <class 'str'>
0c,2018-04-02,03:42:08,0.031,-0.062,-0.937 <class 'str'>
0d,2018-04-02,03:42:09,0.031,-0.061,-0.937 <class 'str'>
0e,2018-04-02,03:42:09,0.031,-0.031,-0.999 <class 'str'>
0f,2018-04-02,03:42:08,0.039,-0.031,-0.999 <class 'str'>
10,2018-04-02,03:42:08,0.032,-0.031,-0.937 <class 'str'>
11,2018-04-02,03:42:10,0.031,-0

94,2018-04-02,03:49:36,0.039,-0.031,-0.999 <class 'str'>
95,2018-04-02,03:49:36,0.039,-0.060,-0.996 <class 'str'>
96,2018-04-02,03:49:36,0.039,-0.058,-0.937 <class 'str'>
97,2018-04-02,03:49:37,0.032,-0.031,-0.999 <class 'str'>
98,2018-04-02,03:49:37,0.039,-0.031,-0.936 <class 'str'>
99,2018-04-02,03:49:37,0.032,-0.061,-0.936 <class 'str'>
9a,2018-04-02,03:49:37,0.037,-0.033,-0.937 <class 'str'>
9b,2018-04-02,03:49:36,0.034,-0.031,-0.936 <class 'str'>
9c,2018-04-02,03:49:38,0.031,-0.031,-0.936 <class 'str'>
9d,2018-04-02,03:49:38,0.039,-0.031,-0.936 <class 'str'>
9e,2018-04-02,03:49:38,0.039,-0.062,-0.937 <class 'str'>
9f,2018-04-02,03:49:38,0.031,-0.029,-0.935 <class 'str'>
a0,2018-04-02,03:49:39,0.031,-0.060,-0.937 <class 'str'>
a1,2018-04-02,03:49:39,0.031,-0.057,-0.991 <class 'str'>
a2,2018-04-02,03:49:39,0.039,-0.062,-0.998 <class 'str'>
a3,2018-04-02,03:49:40,0.032,-0.031,-0.937 <class 'str'>
a4,2018-04-02,03:42:08,0.040,-0.062,-0.998 <class 'str'>
a5,2018-04-02,03:42:08,0.031,-0

28,2018-04-02,03:42:11,0.032,-0.031,-0.937 <class 'str'>
29,2018-04-02,03:42:08,0.039,-0.061,-0.998 <class 'str'>
2a,2018-04-02,03:42:08,0.031,-0.061,-0.937 <class 'str'>
2b,2018-04-02,03:42:40,0.031,-0.062,-0.999 <class 'str'>
2c,2018-04-02,03:42:40,0.031,-0.061,-0.999 <class 'str'>
2d,2018-04-02,03:42:40,0.031,-0.029,-0.999 <class 'str'>
2e,2018-04-02,03:42:40,0.032,-0.031,-0.999 <class 'str'>
2f,2018-04-02,03:42:40,0.031,-0.031,-0.999 <class 'str'>
30,2018-04-02,03:42:41,0.040,-0.030,-0.997 <class 'str'>
31,2018-04-02,03:42:41,0.031,-0.031,-0.937 <class 'str'>
32,2018-04-02,03:42:40,0.039,-0.062,-0.937 <class 'str'>
33,2018-04-02,03:42:40,0.031,-0.031,-0.937 <class 'str'>
34,2018-04-02,03:42:42,0.031,-0.062,-0.998 <class 'str'>
35,2018-04-02,03:42:42,0.039,-0.062,-0.937 <class 'str'>
36,2018-04-02,03:42:42,0.039,-0.031,-0.998 <class 'str'>
37,2018-04-02,03:42:42,0.033,-0.029,-0.937 <class 'str'>
38,2018-04-02,03:42:43,0.031,-0.031,-0.999 <class 'str'>
39,2018-04-02,03:42:43,0.031,-0

c0,2018-04-02,03:42:10,0.038,-0.060,-0.937 <class 'str'>
c1,2018-04-02,03:42:10,0.038,-0.031,-0.937 <class 'str'>
c2,2018-04-02,03:42:10,0.039,-0.029,-0.935 <class 'str'>
c3,2018-04-02,03:42:11,0.033,-0.062,-0.935 <class 'str'>
c4,2018-04-02,03:42:11,0.040,-0.030,-0.937 <class 'str'>
c5,2018-04-02,03:42:11,0.039,-0.061,-0.996 <class 'str'>
c6,2018-04-02,03:42:12,0.031,-0.031,-0.937 <class 'str'>
c7,2018-04-02,03:42:16,0.037,-0.030,-0.935 <class 'str'>
c8,2018-04-02,03:42:16,0.039,-0.061,-0.937 <class 'str'>
c9,2018-04-02,03:42:16,0.032,-0.031,-0.937 <class 'str'>
ca,2018-04-02,03:42:16,0.033,-0.031,-0.998 <class 'str'>
cb,2018-04-02,03:42:16,0.031,-0.061,-0.996 <class 'str'>
cc,2018-04-02,03:42:17,0.033,-0.032,-0.936 <class 'str'>
cd,2018-04-02,03:42:17,0.039,-0.060,-0.936 <class 'str'>
ce,2018-04-02,03:42:17,0.041,-0.031,-0.935 <class 'str'>
cf,2018-04-02,03:42:16,0.032,-0.030,-0.997 <class 'str'>
d0,2018-04-02,03:42:18,0.034,-0.061,-0.999 <class 'str'>
d1,2018-04-02,03:42:18,0.039,-0

In [4]:
read_list[3]
trim_list = read_list[2].replace(" ", ",")

In [5]:
trim_list = read_list[255].split(" ")[1:-1]
print(len(read_list),len(read_list[255].split(" ")[1:-1]))
trim_list

257 16


['ff',
 '5a',
 'c1',
 '28',
 '10',
 '01',
 '00',
 'ff',
 '08',
 'e0',
 '00',
 '00',
 '00',
 '00',
 '00',
 'ff']

In [6]:
timestamp = '0x' + ''.join(trim_list[1:5])
timestamp

'0x5ac12810'

In [7]:
hex_int = int(timestamp, 16)
print(hex_int)
# new_int = hex_int
# print (hex(new_int))

1522608144


In [8]:
time.strftime('%Y-%m-%d,%H:%M:%S', time.localtime(hex_int))

'2018-04-02,03:42:24'

In [9]:
acc_x = trim_list[5]
print("acc_x MSB : ",acc_x)
acc_x = trim_list[6]
print("acc_x LSB : ",acc_x)
acc_x = int(trim_list[5],16) << 8
print("acc_x MSB LShift 8 : " ,acc_x)
acc_x = int(trim_list[5], 16) << 8 | int(trim_list[6], 16)
print("acc_x MSB | LSB : " ,acc_x)
acc_x = (int(trim_list[5], 16) << 8 | int(trim_list[6], 16)) >> 2
print("acc_x MSB | LSB >> 2: " ,acc_x)
acc_x = float(acc_x * 0.000488)
print(acc_x)
# hex_int = int(acc_x, 16)
# print(hex_int)

acc_x MSB :  01
acc_x LSB :  00
acc_x MSB LShift 8 :  256
acc_x MSB | LSB :  256
acc_x MSB | LSB >> 2:  64
0.031232


In [10]:
acc_x = trim_list[7]
print("acc_x MSB : ",acc_x , type(acc_x), len(acc_x))
acc_x = trim_list[8]
print("acc_x LSB : ",acc_x , type(acc_x))



acc_x = int(trim_list[7], 16)
print("acc_x string to int : " ,acc_x , type(acc_x), sys.getsizeof(acc_x), '{0:b}'.format(acc_x))

acc_x = int(trim_list[8], 16)
print("acc_x string to int : " ,acc_x , type(acc_x), sys.getsizeof(acc_x), '{0:b}'.format(acc_x))

acc_x = int(trim_list[7], 16) << 8
print("acc_x MSB LShift 8 : " ,acc_x , type(acc_x), sys.getsizeof(acc_x), '{0:b}'.format(acc_x))

acc_x = acc_x | int(trim_list[8], 16)
#acc_x = int(trim_list[7], 16) << 8 | int(trim_list[8], 16)
print("acc_x MSB | LSB : " ,acc_x , type(acc_x), '{0:b}'.format(acc_x))

a = _twos_comp(acc_x >> 2 , 14)
print(a , type(a), float(a) * 0.000488)

acc_x = acc_x >> 2 
#acc_x = (int(trim_list[7], 16) << 8 | int(trim_list[8], 16)) >> 2
print("acc_x MSB | LSB >> 2: " ,acc_x , type(acc_x), bin(acc_x))
acc_x = acc_x * 0.000488
#acc_x = float(acc_x ) * 0.000488
print(acc_x , type(acc_x), '{0:0.3f}'.format(acc_x))
# hex_int = int(acc_x, 16)
# print(hex_int)

acc_x MSB :  ff <class 'str'> 2
acc_x LSB :  08 <class 'str'>
acc_x string to int :  255 <class 'int'> 28 11111111
acc_x string to int :  8 <class 'int'> 28 1000
acc_x MSB LShift 8 :  65280 <class 'int'> 28 1111111100000000
acc_x MSB | LSB :  65288 <class 'int'> 1111111100001000
-62 <class 'int'> -0.030255999999999998
acc_x MSB | LSB >> 2:  16322 <class 'int'> 0b11111111000010
7.965136 <class 'float'> 7.965


In [11]:
12.5 - 45.2

-32.7

In [12]:
acc_x = (((int(trim_list[5], 16) << 8 | int(trim_list[6], 16))))
acc_y = (((int(trim_list[7], 16) << 8 | int(trim_list[8], 16))))
acc_z = (((int(trim_list[9], 16) << 8 | int(trim_list[10], 16))))

a = _twos_comp(acc_x >> 2 , 14) * 0.000488
b = _twos_comp(acc_y >> 2 , 14) * 0.000488
c = _twos_comp(acc_z >> 2 , 14) * 0.000488

print(a , type(a))
print(b , type(b))
print(c , type(c))
print('{0:0.3f}, {1:0.3f}, {2:0.3f}'.format(a, b, c ))

acc_x = float((((int(trim_list[5], 16) << 8 | int(trim_list[6], 16)) >> 2) )) * 0.000488 
acc_y = float((((int(trim_list[7], 16) << 8 | int(trim_list[8], 16)) >> 2) )) * 0.000488
acc_z = float((((int(trim_list[9], 16) << 8 | int(trim_list[10], 16)) >> 2))) * 0.000488
print(acc_x)
print(acc_y)
print(acc_z)
#acc_x = float(acc_x * 0.000488)

0.031232 <class 'float'>
-0.030255999999999998 <class 'float'>
-0.999424 <class 'float'>
0.031, -0.030, -0.999
0.031232
7.965136
6.9959679999999995


In [13]:
s.close()

In [14]:
list_length = len(read_list)
list_length

257

In [15]:
for i in range(list_length - 1):
    
    trim_list = read_list[i].split(" ")[1:-1]
#     print(len(read_list),len(read_list[i].split(" ")[1:-1]))
    timestamp = '0x' + ''.join(trim_list[1:5])
    hex_int = int(timestamp, 16)
#     print(hex_int)
    datetime = time.strftime('%Y-%m-%d,%H:%M:%S', time.localtime(hex_int))
    
    acc_x = (((int(trim_list[5], 16) << 8 | int(trim_list[6], 16))))
    acc_y = (((int(trim_list[7], 16) << 8 | int(trim_list[8], 16))))
    acc_z = (((int(trim_list[9], 16) << 8 | int(trim_list[10], 16))))

    a = _twos_comp(acc_x >> 2 , 14) * 0.000488
    b = _twos_comp(acc_y >> 2 , 14) * 0.000488
    c = _twos_comp(acc_z >> 2 , 14) * 0.000488
    
#     print(a , type(a))
#     print(b , type(b))
#     print(c , type(c))
    result_string = datetime + " : " +  '{0:0.3f}, {1:0.3f}, {2:0.3f}'.format(a, b, c )
    print(result_string , type(result_string))
    
   

2018-04-02,03:42:24 : 0.000, -0.030, -0.999 <class 'str'>
2018-04-02,03:42:24 : 0.035, -0.031, -0.935 <class 'str'>
2018-04-02,03:42:25 : 0.000, -0.031, -0.937 <class 'str'>
2018-04-02,03:42:24 : 0.032, -0.062, -0.997 <class 'str'>
2018-04-02,03:42:26 : 0.031, -0.061, -0.999 <class 'str'>
2018-04-02,03:42:26 : 0.000, -0.093, -0.998 <class 'str'>
2018-04-02,03:42:26 : 0.000, -0.094, -0.937 <class 'str'>
2018-04-02,03:42:08 : 0.039, -0.094, -0.937 <class 'str'>
2018-04-02,03:42:08 : 0.000, -0.093, -0.998 <class 'str'>
2018-04-02,03:42:08 : 0.000, -0.061, -0.935 <class 'str'>
2018-04-02,03:42:08 : 0.031, -0.062, -0.936 <class 'str'>
2018-04-02,03:42:08 : 0.031, -0.061, -0.937 <class 'str'>
2018-04-02,03:42:08 : 0.031, -0.062, -0.937 <class 'str'>
2018-04-02,03:42:09 : 0.031, -0.061, -0.937 <class 'str'>
2018-04-02,03:42:09 : 0.031, -0.031, -0.999 <class 'str'>
2018-04-02,03:42:08 : 0.039, -0.031, -0.999 <class 'str'>
2018-04-02,03:42:08 : 0.032, -0.031, -0.937 <class 'str'>
2018-04-02,03:

In [16]:
def _twos_comp(val, bits):
    # Convert an unsigned integer in 2's compliment form of the specified bit
    # length to its signed integer value and return it.
    if val & (1 << (bits - 1)) != 0:
        return val - (1 << bits)
    return val

In [17]:
i = _twos_comp(57364 >> 2, 14)
print(i, bin(i), i * 0.000488)
'{0:0.3f}'.format(i * 0.000488)

-2043 -0b11111111011 -0.996984


'-0.997'

In [18]:
read_list[256][0]

'e'

In [19]:
values = 57364
print(values , bin(values))
values = values >> 2 
print(values, bin(values))
print(values & (1 << (13)), bin(values & (1 << (14 - 1))))
tmp = (1 << 14)
print(tmp , bin(tmp))
values = values - tmp
print(values, bin(values))
#val & (1 << (bits - 1))

57364 0b1110000000010100
14341 0b11100000000101
8192 0b10000000000000
16384 0b100000000000000
-2043 -0b11111111011
